In [115]:
import numpy as np
import pandas as pd

In [231]:
ds=pd.read_csv('marketing_tr_3.csv')

In [232]:
ds=ds.drop(['marital','schooling','default','housing','loan','campaign','previous','poutcome','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','pmonths'],axis=1)

In [233]:
print ds

      custAge     profession    contact month day_of_week  pdays  pastEmail  \
0        55.0         admin.   cellular   nov         mon    999          0   
1         NaN    blue-collar   cellular   jul         mon    999          0   
2        42.0     technician  telephone   may         mon    999          0   
3        55.0     management   cellular   jul         wed    999          0   
4         NaN         admin.   cellular   may         tue    999          1   
5        47.0    blue-collar  telephone   may         fri    999          0   
6        35.0       services  telephone   may         tue    999          0   
7         NaN       services  telephone   may         wed    999          0   
8        28.0     unemployed   cellular   may         fri    999          0   
9        46.0    blue-collar   cellular   nov         mon    999          0   
10        NaN         admin.   cellular   aug         mon    999          0   
11       41.0    blue-collar   cellular   jul       

In [234]:
ds['profession'].unique()

array(['admin.', 'blue-collar', 'technician', 'management', 'services',
       'unemployed', 'housemaid', 'unknown', 'self-employed', 'retired',
       'student', 'entrepreneur'], dtype=object)

In [235]:
j=1
profession={}
for i in ds['profession'].unique():
    ds.loc[ds['profession']==i,'profession']=j
    profession[i]=j
    j+=1

In [236]:
print profession

{'management': 4, 'retired': 10, 'self-employed': 9, 'unknown': 8, 'unemployed': 6, 'housemaid': 7, 'admin.': 1, 'services': 5, 'student': 11, 'technician': 3, 'entrepreneur': 12, 'blue-collar': 2}


In [237]:
ds.loc[ds['responded']=='no','responded']=0
ds.loc[ds['responded']=='yes','responded']=1

In [238]:
ds['custAge']=ds['custAge'].fillna(ds['custAge'].median())

In [239]:
ds.loc[ds['contact']=='cellular','contact']=0
ds.loc[ds['contact']=='telephone','contact']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='feb','month']=2
ds.loc[ds['month']=='mar','month']=3
ds.loc[ds['month']=='apr','month']=4
ds.loc[ds['month']=='may','month']=5
ds.loc[ds['month']=='jun','month']=6
ds.loc[ds['month']=='jul','month']=7
ds.loc[ds['month']=='aug','month']=8
ds.loc[ds['month']=='sep','month']=9
ds.loc[ds['month']=='oct','month']=10
ds.loc[ds['month']=='nov','month']=11
ds.loc[ds['month']=='dec','month']=12

In [240]:
ds['day_of_week']=ds['day_of_week'].fillna(0)

In [241]:
ds.loc[ds['day_of_week']=='mon','day_of_week']=1
ds.loc[ds['day_of_week']=='tue','day_of_week']=2
ds.loc[ds['day_of_week']=='wed','day_of_week']=3
ds.loc[ds['day_of_week']=='thu','day_of_week']=4
ds.loc[ds['day_of_week']=='wed','day_of_week']=5
ds.loc[ds['day_of_week']=='fri','day_of_week']=6
ds.loc[ds['day_of_week']=='sat','day_of_week']=7

In [242]:
train= np.array(ds)

In [243]:
print train.shape

(7414, 8)


In [246]:
print train[:,-1]

[1 0 0 ..., 1 0 0]


In [247]:
print np.sum(train[:,-1])

2999


In [248]:
ds.head()

,custAge,profession,contact,month,day_of_week,pdays,pastEmail,responded
0,55.0,1,0,11,1,999,0,1
1,38.0,2,0,7,1,999,0,0
2,42.0,3,1,5,1,999,0,0
3,55.0,4,0,7,3,999,0,0
4,38.0,1,0,5,2,999,1,1


In [249]:
from keras.utils import np_utils

In [256]:
split=0.80*train.shape[0]
train_data=train[:,:-1]

X_train=train_data[:split,:]
X_test=train_data[split:,:]

labels = np.asarray(train[:,-1], dtype="|S6")
labels=np_utils.to_categorical(labels)

y_train=labels[:split,:]
y_test=labels[split:,:]

print X_train.shape,X_test.shape
print y_train.shape, y_test.shape


(5931, 7) (1483, 7)
(5931, 2) (1483, 2)


/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/apple/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [154]:
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam

In [257]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim = X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.75))
model.add(Dense(2, activation='softmax'))
optimizer=Adam(lr=0.01)
model.summary()
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_33 (Dense)                 (None, 32)            256         dense_input_9[0][0]              
____________________________________________________________________________________________________
dense_34 (Dense)                 (None, 64)            2112        dense_33[0][0]                   
____________________________________________________________________________________________________
batchnormalization_9 (BatchNorma (None, 64)            256         dense_34[0][0]                   
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 64)            0           batchnormalization_9[0][0]       
___________________________________________________________________________________________

In [258]:
hist=model.fit(X_train, y_train,
          nb_epoch=15,
          shuffle=True,
               batch_size=100,
          validation_data=(X_test,y_test))

Train on 5931 samples, validate on 1483 samples
Epoch 1/15
5931/5931 [==============================] - 0s - loss: 0.6132 - acc: 0.6791 - val_loss: 2.8503 - val_acc: 0.6224
Epoch 2/15
5931/5931 [==============================] - 0s - loss: 0.5655 - acc: 0.7382 - val_loss: 2.4601 - val_acc: 0.6224

In [176]:
print train[0,:].dtype


object


In [173]:
test=np.array(ds)
test=test[5000:7000,:]


In [189]:
print train[4,:]

[38.0 1 0 5 2 999 1 1]


In [192]:
test=np.array(([38.0, 1, 0, 5, 2, 999, 1]))
test=test.reshape((1,7))
print test.shape

(1, 7)


In [193]:
pred = model.predict([test])

In [194]:
# pred= (pred == pred.max(axis=1)[:,None]).astype(int)
print pred


[[  1.00000000e+00   2.80115070e-13]]


In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_out = layer_dict['block4_conv2'].output


In [107]:
model.save('nn.h5')

In [106]:
from keras.models import load_model
model_pred=load_model('nn.h5')

AttributeError: 'History' object has no attribute 'predict'